## Populate an RDF database

This notebook reports the main steps to download CSV files, process them and create an RDF dataset from them accordingly to an ontology. 

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

In [385]:
# required libraries
import pandas as pd
import os
import ast
import unicodedata
import hashlib
import re
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [409]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.parent.absolute())
print(path)
grammyUrl = path + '/csv/the_grammy_awards_mapped.csv'
print(grammyUrl)
albumsUrl = path + '/csv/musicoset_metadata/albums.csv'
print(albumsUrl)
songsUrl = path + '/csv/musicoset_metadata/albums.csv'
print(songsUrl)
artistsUrl = path + '/csv/musicoset_metadata/albums.csv'
print(artistsUrl)
tracksUrl = path + '/csv/musicoset_metadata/tracks.csv'
print(tracksUrl)

# saving folder
savePath =  path + '/PopulateRDFdb/PopulateGrammyCategories/'

c:\Users\fgall\Desktop\MELODY
c:\Users\fgall\Desktop\MELODY/csv/the_grammy_awards_mapped.csv
c:\Users\fgall\Desktop\MELODY/csv/musicoset_metadata/albums.csv
c:\Users\fgall\Desktop\MELODY/csv/musicoset_metadata/albums.csv
c:\Users\fgall\Desktop\MELODY/csv/musicoset_metadata/albums.csv
c:\Users\fgall\Desktop\MELODY/csv/musicoset_metadata/tracks.csv


## Grammy & candidates/winners (Songs, Artists, Albums)

In [387]:
# Load the CSV files in memory
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
grammy = pd.read_csv(grammyUrl, sep=',', keep_default_na=False, na_values=['_'])

In [388]:
album = pd.read_csv(albumsUrl, sep='\t', index_col='album_id', keep_default_na=False, na_values=['_'])
album.info()
# Lista per salvare le coppie GrammyID e album_id
matched_pairs_grammy_album = []

<class 'pandas.core.frame.DataFrame'>
Index: 26519 entries, 5n1GSzC1Reao29ScnpLYqp to 6wf7Rh10EoTaqZMdN2xRlI
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          26519 non-null  object
 1   billboard     26519 non-null  object
 2   artists       26519 non-null  object
 3   popularity    26519 non-null  int64 
 4   total_tracks  26519 non-null  int64 
 5   album_type    26519 non-null  object
 6   image_url     26519 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.6+ MB


In [389]:
grammy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6323 entries, 0 to 6322
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      6323 non-null   int64 
 1   category  6323 non-null   object
 2   nominee   6323 non-null   object
 3   workers   6323 non-null   object
 4   winner    6323 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 203.9+ KB


We need to install <code>RDFLib</code>

<code>pip3 install rdflib </code> [Documentation](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)

In [390]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD, SKOS, RDFS


In [391]:
a = "MGMT"
b = "M.G.M.B."
fuzz.ratio(a,b)


50

In [392]:
# Construct the country and the movie ontology namespaces not known by RDFlib
ME = Namespace("http://www.dei.unipd.it/~gdb/ontology/melody/")



#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("mel", ME)
g.bind("skos", SKOS)
g.bind("rdfs", RDFS)

In [393]:
# ID univoco nel formato YYYY_CAT_HASH
def create_grammy_id(year, category, title, artist, is_winner):
    
    year = str(year)
    category = str(category) if pd.notna(category) else ''
    title = str(title) if pd.notna(title) else ''
    artist = str(artist) if pd.notna(artist) else ''
    is_winner = str(is_winner) if pd.notna(is_winner) else ''

    # Pulizia e normalizzazione dei dati
    def clean_text(text):
        # Rimuove caratteri speciali e converte in lowercase
        return re.sub(r'[^\w\s-]', '', text).lower().strip()
    
    # Crea una stringa concatenata con tutti i dati
    full_string = f"{year}_{clean_text(category)}_{clean_text(title)}_{clean_text(artist)}_{is_winner}"
    
    # Genera un hash SHA-256 troncato
    hash_object = hashlib.sha256(full_string.encode())
    short_hash = hash_object.hexdigest()[:8]
    
    # Crea l'ID finale
    category_abbr = ''.join(word[0] for word in clean_text(category).split()[:3])
    final_id = f"{year}_{category_abbr}_{short_hash}"
    
    return final_id


def normalize_uri(name):
    # Rimuove accenti e caratteri speciali
    name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    # Sostituisce spazi con trattini
    name = name.replace(" ", "-")
    # Rimuove virgole e apostrofi
    name = name.replace(",", "").replace("'", "")
    return name

# Funzione per estrarre gli artisti (vettorizzabile)
def extract_artist_name_fast(artists_column):
    return artists_column.str.extract(r"'[^']*': '([^']*)'")[0]  # Estrai direttamente il nome

In [394]:
%%time 
#measure execution time

#iterate over the grammy dataframe
for index, row in grammy.iterrows():

    

    grammy_id = create_grammy_id(
        row['year'],       
        row['category'],
        row['nominee'],
        row['workers'],
        row['winner']
    )


    # Create the node to add to the Graph
    # the node has the namespace + the movie id as URI
    current_grammy = URIRef(ME[grammy_id])

    if False:
        print(row)
        print(grammy_id)
        print(ME.Grammy) # http://www.dei.unipd.it/~gdb/ontology/melody/Grammy
        print(current_grammy) # http://www.dei.unipd.it/~gdb/ontology/melody/1959_a_a98ae627

    # Add triples using store's add() method.
    g.add((current_grammy, RDF.type, ME.Grammy))
    g.add((current_grammy, ME['hasCategory'], Literal(normalize_uri(row['category']))))
    g.add((current_grammy, ME['year'], Literal(row['year'], datatype=XSD.gYear)))

    if "album" in row['category'].lower():

        # Estrarre solo i nomi degli artisti da album['artists']
        album['artist_name'] = album['artists'].apply(lambda x: list(eval(x).values())[0])  # Convertire la stringa in dizionario e prendere il valore

        isWinner = isinstance(row['winner'], bool)

        # Pulire row['workers'] e ottenere una lista di nomi
        worker_names = re.sub(r'\([^)]*\)', '', row['workers'])  # Rimuove il contenuto tra parentesi
        worker_names = [name.strip() for name in worker_names.split(',')]  # Divide per virgola e rimuove gli spazi

        # Filtrare gli album che corrispondono per titolo e artista
        matched_album = album[
            (album['billboard'].str.lower() == row['nominee'].lower()) &  # Confronta il titolo
            (album['artist_name'].str.lower().isin([name.lower() for name in worker_names]))  # Confronta l'artista
        ]

        # Stampare tutte le corrispondenze
        if not matched_album.empty:
            for album_id in matched_album.index:
                print(grammy_id, album_id, isWinner)
                matched_pairs_grammy_album.append((grammy_id, album_id, isWinner))


 




1965_b_47f18fcb 381STOo2xuzaUPS3GaGiwi True
1966_b_683d611b 0eBIK8q43cUJ4SiYCoDVNu True
1966_b_942e6a47 4wXA2W3Ody5SO5wKp2Y4Rk True
1966_b_ffb53c16 27SMV8TOEzD5NvCI6dK7Xc True
1967_a_00c07763 6lmuhQkwg4qkjytuho7Oxz True
1967_b_a1d5709b 41q01T7MnMnWy3a3Swa63g True
1968_a_532411e2 6QaVfG1pHYl1z15ZxkvVDW True
1968_b_4d82d10a 5Ju2gORF8m4VA9TZn006kv True
1968_b_122f5ebf 6QaVfG1pHYl1z15ZxkvVDW True
1968_b_23a98b28 7b4LQX82Gnt2kZ9I5MUZil True
1969_a_401257aa 29tTA46kurlOioRkjBqOMS True
1969_b_914e57bf 4TJIdlY9hGSSTO1kUs1neh True
1969_b_97a28cbf 056tSaBR2WyN1nnmfIzkEE True
1969_b_4ef5ad8f 4ALVyY1OJCf30LCBwhkzOd True
1970_b_b329a3f9 5WBx64FIN04CvM2T1MGrUN True
1970_b_9196d87d 0ETFjACtuP2ADo6LFhL6HN True
1970_b_b5cef7ce 03iFLgmgkLT7X5gnXVPID5 True
1970_b_ed61bcaa 0o2ZKR3DbPg23bt11WiWhS True
1971_a_31f80f30 0JwHz5SSvpYWuuCNbtYZoV True
1971_b_aff4800c 0JwHz5SSvpYWuuCNbtYZoV True
1971_b_106e155e 1wVO8nHzgcim0IBzbXnYX0 True
1972_a_c4e6b0ec 12n11cgnpjXKLeqrnIERoS True
1972_b_25fc0485 243DiQm4D8A679lJ

In [395]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'grammy.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
CPU times: total: 359 ms
Wall time: 457 ms


# Referential integrity
Note that in RDF we are in an open world situation. We cannot guarantee the referential integrity between the entities. 

## Album

Let us generate the RDF data relative to the movie workers.

In [422]:
album = pd.read_csv(albumsUrl, sep='\t', index_col='album_id', keep_default_na=False, na_values=['_'])
album.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26519 entries, 5n1GSzC1Reao29ScnpLYqp to 6wf7Rh10EoTaqZMdN2xRlI
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          26519 non-null  object
 1   billboard     26519 non-null  object
 2   artists       26519 non-null  object
 3   popularity    26519 non-null  int64 
 4   total_tracks  26519 non-null  int64 
 5   album_type    26519 non-null  object
 6   image_url     26519 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.6+ MB


In [423]:
tracks = pd.read_csv(tracksUrl, sep='\t', index_col='album_id', keep_default_na=False, na_values=['_'])
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20405 entries, 2fYhqwDWXjbpjaIJPEfKFw to 3pBArpt3QcnvVj58hl6Ghe
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   song_id                 20405 non-null  object
 1   track_number            20405 non-null  int64 
 2   release_date            20405 non-null  object
 3   release_date_precision  20405 non-null  object
dtypes: int64(1), object(3)
memory usage: 797.1+ KB


People are modeled with the FOAF ontology. 
Refer to [FOAF Documentation](http://xmlns.com/foaf/spec/)

In [424]:
#create a new graph
g = Graph()
ME = Namespace("http://www.dei.unipd.it/~gdb/ontology/melody/")
g.bind("xsd", XSD)
g.bind("mel", ME)
g.bind("skos", SKOS)
g.bind("rdfs", RDFS)

In [ ]:
%%time 
#measure execution time


#iterate over the album dataframe
for index, row in albums.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the person id as URI
    current_album = URIRef(ME[index])

    if False:
        print(row)
        print(index) #5n1GSzC1Reao29ScnpLYqp
        print(ME.Album) # http://www.dei.unipd.it/~gdb/ontology/melody/Album
        print(current_album) # http://www.dei.unipd.it/~gdb/ontology/melody/5n1GSzC1Reao29ScnpLYqp

    g.add((current_album, RDF.type, ME.Album))
    # Add triples using store's add() method.
    g.add((current_album, ME['name'], Literal(row['name'], datatype=XSD.string)))
    if row['total_tracks'] is not None and isinstance(row['total_tracks'], int):
        g.add((current_album, ME['totalTracks'], Literal(row['total_tracks'], datatype=XSD.positiveInteger)))
    else:
         g.add((current_album, ME['totalTracks'], Literal(0, datatype=XSD.positiveInteger)))


    print(index)
    album_tracks = tracks[tracks.index == index]
    

    #g.add((current_album, ME['containsSong'], current_song))
        
    #print(f"Album {index} contiene la canzone {song_id}")


    for grammy_id, album_id, winner in matched_pairs_grammy_album:
        if album_id == index:
            current_album = URIRef(ME[album_id])
            current_grammy = URIRef(ME[grammy_id])

            if (winner):
                g.add((current_album, ME['winner'], current_grammy))
                print(f"{album_id} won {grammy_id}")
            else:
                g.add((current_album, ME['candidated'], current_grammy))
                print(f"{album_id} lost {grammy_id}")

    





    
    

5n1GSzC1Reao29ScnpLYqp
6UYZEYjpN1DYRW0kqFy9ZE
7uVimUILdzSZG4KKKWToq0
35s58BRTGAEWztPo9WqCIs
41GuZcammIkupMPKH2OJ6I
41GuZcammIkupMPKH2OJ6I won 2019_b_9f73df9b
6thZNGX8hUVSjUrqJgPB9b


KeyError: '6thZNGX8hUVSjUrqJgPB9b'

In [406]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'albums.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 1.89 s
Wall time: 2.15 s


## Person-Movie Join

In [401]:
# Load the CSV files in memory
join = pd.read_csv(joinTableUrl, sep=',', index_col='imdb_title_id', keep_default_na=False, na_values=['_'])

NameError: name 'joinTableUrl' is not defined

In [ ]:
#create a new graph
g = Graph()

In [ ]:
#regular expressions
import re
actor = re.compile('act*')

In [ ]:
%%time 
#measure execution time

#iterate over the join table dataframe
for index, row in join.iterrows():
    # Create the node about the movie
    # note that we do not add this resource to the database (created before)
    Movie = URIRef(MO[index])
    
    # Create the node about the person
    # note that we do not add this resource to the database (created before)
    Person = URIRef(MO[row['imdb_name_id']])
    # get the role of the person
    role = row['category']
    
    # we have an actor or actress
    if actor.match(role): 
        g.add((Person, MO['acted'], Movie))
    elif (role=='director'):
        g.add((Person, MO['directed'], Movie))
    else:
        # note that, with the defined ontology, we cannot caracterize the specific role of this person in the movie. 
        # why?
        g.add((Person, MO['worked'], Movie))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("mo", MO)

KeyboardInterrupt: 

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'name_movie_join.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 1min 20s, sys: 1.57 s, total: 1min 22s
Wall time: 1min 23s


## Awards - Oscars data
Note that if we do not check the referential integrity then we could produce ghost triple movie-nominee-oscar where the movie is not in the RDF graph.

On the other hand, we can check if an actor or a movie exists by using the DataFrame in Python. Note that this is an external check and not a constraints met by the RDF DB.



In [ ]:
# Load the CSV files in memory
oscars = pd.read_csv(oscarsUrl, sep=',', keep_default_na=False, na_values=['_'])

In [ ]:
from num2words import num2words
import string
import re
#create a new graph
g = Graph()

In [ ]:
%%time
#iterate over the join table dataframe
for index,row in oscars.iterrows():
    #create the oscar with a custom id 
    cat = re.sub(r'[^\w\s]','',row['category'])
    Oscar = URIRef(MO['oscar_'+cat.replace(" ", "").lower()+'_'+ str(num2words(row['ceremony'], to='ordinal'))])
    
    # check if there already is at least a triple about this oscar
    if not (Oscar, None, None) in g:    
        # check if the oscar is already in the graph
        g.add((Oscar, RDF.type, MO.Oscar))
        g.add((Oscar, MO['category'], Literal(row['category'].lower(), datatype=XSD.string)))
        g.add((Oscar, MO['year'], Literal(row['year_ceremony'], datatype=XSD.gYear)))
    
    # check if there is a name matching the people, meaning that the oscar can be associated to a person
    if (people["name"] == row["name"]).any() == True :
        #there is a person with this name
        # Create the node about the person
        # note that we do not add this resource to the database (created before)
        Person = URIRef(MO[people[people["name"]==row["name"]].index[0]])
        if row['winner']:
            g.add((Person, MO['winner'], Oscar))
        else:
            g.add((Person, MO['nominated'], Oscar))
    
    # an oscar for a person is also to be considered an oscar for the movie
    # check if the movie is in our DB
    if (movies["original_title"] == row["film"]).any():
        # there is a movie with this title
        Movie = URIRef(MO[movies[movies["original_title"]==row["film"]].index[0]])
        if row['winner']:
            g.add((Movie, MO['winner'], Oscar))
        else:
            g.add((Movie, MO['nominated'], Oscar))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("mo", MO)

CPU times: user 4min 53s, sys: 4.85 s, total: 4min 58s
Wall time: 5min 8s


In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'oscars.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 2.88 s, sys: 60.9 ms, total: 2.94 s
Wall time: 4.3 s
